In [157]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import branca

In [93]:
def get_geojson_grid(upper_right, lower_left, n_x, n_y):
    """Returns a grid of geojson rectangles, and computes the exposure 
        in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is 
   the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is 
     the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n_x+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n_y+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]
            
            # Define dimensions of box in grid
            upper_left = [lat + lat_stride, lon]
            upper_right = [lat + lat_stride, lon + lon_stride]
            lower_right = [lat, lon + lon_stride]
            lower_left = [lat, lon]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes 

In [125]:

ascii_grid = np.loadtxt("/Users/shuffle_new/Desktop/Prelim/Modeling/Data/gpw-v4-population-density-rev11_2020_30_sec_asc/gpw_v4_population_density_rev11_2020_30_sec_1.asc", skiprows=30)

In [169]:
# upper_right = [38.17,-121.89]
# lower_left = [37,-123.27]


upper_right = [38.17,-121.89]
lower_left = [38,-122.27]

In [170]:
np.shape(ascii_grid)

(10776, 10800)

In [171]:
unit_x = 90/np.shape(ascii_grid)[0]
unit_y = 90/np.shape(ascii_grid)[1]

In [ ]:
def lat_to_id(lat):
    return int(np.shape(ascii_grid)[0] - upper_right[0]/unit_x)

In [172]:
upper_right_id = [int(np.shape(ascii_grid)[0] - upper_right[0]/unit_x),
                  int((upper_right[1]+180)/unit_y)]
lower_left_id = [int(np.shape(ascii_grid)[0] - lower_left[0]/unit_x),
                 int((lower_left[1]+180)/unit_y)]

upper_right_real = [90-upper_right_id[0]*unit_x,
                    upper_right_id[1]*unit_y-180]
lower_left_real = [90-lower_left_id[0]*unit_x,
                    lower_left_id[1]*unit_y-180]

In [173]:
upper_right_id

[6205, 6973]

In [174]:
lower_left_id

[6226, 6927]

In [175]:
def plot_density(n_x, n_y):
    # Creating a grid of nxn from the given cordinate corners     
    grid = get_geojson_grid(upper_right_real, lower_left_real , n_x, n_y)
    # Holds number of points that fall in each cell & time window if provided
    counts_array = np.zeros(n_x * n_y)
    
    # Adding the total number of visits to each cell
    i = 0
    for box in grid:
        
        # get the corners for each cell
        upper_right = box["properties"]["upper_right"]
        lower_left = box["properties"]["lower_left"]
#         print(upper_right[0])
        counts_array[i] = ascii_grid[int(upper_right[0]),int(upper_right[1])]
        i += 1
        
    m = folium.Map(zoom_start = 10, location=[38, -121])
    # Add GeoJson to map
    for i, geo_json in enumerate(grid):
        relativeCount = counts_array[i]*100/4345
        color = plt.cm.YlGn(relativeCount)
        color = mpl.colors.to_hex(color)
        gj = folium.GeoJson(geo_json,
                style_function=lambda feature, color=color: {
                    'fillColor': color,
                    'color':"gray",
                    'weight': 0.5,
                    'dashArray': '6,6',
                    'fillOpacity': 0.8,
                })
        m.add_child(gj)
        
    colormap = branca.colormap.linear.YlGn_09.scale(0, 1)
    colormap = colormap.to_step(index=[0, 0.3, 0.6, 0.8 , 1])
    colormap.caption = 'Relative density of fleet activity per cell'
    colormap.add_to(m)
    return m
#     return counts_array

In [176]:
n_x = lower_left_id[0] - upper_right_id[0]
n_y = upper_right_id[1] - lower_left_id[1]
tmp = plot_density(n_x, n_y)

In [177]:
tmp